In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input

데이터 로드 및 확인

In [3]:
# 데이터 로드
df = pd.read_csv('seeds_dataset.txt', sep='\s+', header=None)

# 데이터 확인
df

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


레이블 및 전처리

In [4]:
# 특성과 타겟 분리
X = df.drop(columns=[7])  # 0~6번 열: 특성
y = df[7]  # 7번 열: 클래스 (1, 2, 3)

# 클래스 라벨을 0부터 시작하도록 조정 (1 → 0, 2 → 1, 3 → 2)
y = y - 1

# 원-핫 인코딩
y_one_hot = to_categorical(y, num_classes=3)

훈련 데이터

In [5]:
# 3. 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

데이터 스케일링

In [6]:
# 4. 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Sequential API 방법

In [12]:
model_seq = Sequential()
model_seq.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model_seq.add(Dense(32, activation='relu'))
model_seq.add(Dense(3, activation='softmax'))

model_seq.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

model_seq.fit(X_train, y_train, epochs=50, batch_size=16,
              validation_split=0.2, callbacks=[early_stop])

# 평가
loss, acc = model_seq.evaluate(X_test, y_test)
print(f"Sequential 모델 정확도: {acc:.4f}")

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2002 - loss: 1.1530 - val_accuracy: 0.5882 - val_loss: 0.8924
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6674 - loss: 0.8976 - val_accuracy: 0.9118 - val_loss: 0.6811
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8919 - loss: 0.7175 - val_accuracy: 0.9118 - val_loss: 0.5464
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8822 - loss: 0.5890 - val_accuracy: 0.9412 - val_loss: 0.4484
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9264 - loss: 0.5111 - val_accuracy: 0.9412 - val_loss: 0.3757
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9409 - loss: 0.4286 - val_accuracy: 0.9412 - val_loss: 0.3186
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9251 - loss: 0.3851 - val_accuracy: 0.9118 - val_loss: 0.2754
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9249 - loss: 0.3157 - val_accuracy: 0.9118 - val_loss: 0.2410


Functional API 버전

In [13]:
inputs = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)

model_func = Model(inputs=inputs, outputs=outputs)
model_func.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

model_func.fit(X_train, y_train, epochs=50, batch_size=16,
               validation_split=0.2, callbacks=[early_stop])

# 평가
loss, acc = model_func.evaluate(X_test, y_test)
print(f"Functional API 모델 정확도: {acc:.4f}")


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.2824 - loss: 1.1119 - val_accuracy: 0.8529 - val_loss: 0.8328
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8871 - loss: 0.7956 - val_accuracy: 0.9118 - val_loss: 0.5747
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8808 - loss: 0.6187 - val_accuracy: 0.9118 - val_loss: 0.4365
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8803 - loss: 0.4887 - val_accuracy: 0.9118 - val_loss: 0.3537
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8766 - loss: 0.3946 - val_accuracy: 0.9412 - val_loss: 0.3019
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8706 - loss: 0.3787 - val_accuracy: 0.9412 - val_loss: 0.2615
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9046 - loss: 0.3319 - val_accuracy: 0.9412 - val_loss: 0.2333
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8922 - loss: 0.3055 - val_accuracy: 0.9412 - val_loss: 0.2110
Ep

Subclassing API 버전

In [ ]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(32, activation='relu')
        self.out = Dense(3, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.out(x)

model_sub = MyModel()
model_sub.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

model_sub.fit(X_train, y_train, epochs=50, batch_size=16,
              validation_split=0.2, callbacks=[early_stop])

# 평가
loss, acc = model_sub.evaluate(X_test, y_test)
print(f"Subclassing 모델 정확도: {acc:.4f}")


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.2982 - loss: 1.1107 - val_accuracy: 0.6471 - val_loss: 0.8773
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5904 - loss: 0.9001 - val_accuracy: 0.8235 - val_loss: 0.7000
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7589 - loss: 0.7527 - val_accuracy: 0.8529 - val_loss: 0.5708
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8401 - loss: 0.6421 - val_accuracy: 0.9118 - val_loss: 0.4790
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8784 - loss: 0.5720 - val_accuracy: 0.9118 - val_loss: 0.4040
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8626 - loss: 0.4911 - val_accuracy: 0.9412 - val_loss: 0.3517
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8935 - loss: 0.4342 - val_accuracy: 0.9412 - val_loss: 0.3114
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8881 - loss: 0.3946 - val_accuracy: 0.9412 - val_loss: 0.2801
E